In [7]:
# #!/usr/bin/python
# -*- coding: UTF-8 -*-
### 引入所有包,如果缺少某个包，包的名字附于之后
import numpy as np
# numpy

import pandas as pd
# pandas

import requests
# requests

from bs4 import BeautifulSoup
# BeautifulSoup

import re
# re

from fake_useragent import UserAgent
# fake-useragent

import json
# json

import time
# time

import random
# random

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
# selenium

from PIL import Image,ImageEnhance
# PIL

import hashlib
# hashlib

from collections import Counter
# collections

from gensim import corpora, models, similarities
# gensim 

import codecs, sys
# codecs

import os
# os

import shutil
# shutil

import jieba
# jieba

from pyhanlp import *
# pyhanlp，注意hanlp需要java的工具

import jpype
# jpype

import tensorflow as tf
# tensorflow

import matplotlib.pyplot as plt
# matplotlib

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer
# sklearn

from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams
from pdfminer.pdfparser import PDFParser, PDFDocument
from pdfminer.pdfdevice import PDFDevice
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
# pdfminer3k

from tkinter import *
from tkinter import filedialog
## tk文件导入模块

In [14]:
def import_table(root):
    """
    return table"选中的表格
    """
    try:
        root.filename = filedialog.askopenfilename(filetypes=(("xlsx", "*.xlsx"),("xlsx", "*.xlsx")))        
        if ".xlsx" in root.filename:
            ### 该目录下有该文件
            table = pd.read_excel(root.filename)
            root.destroy()
            return table
    except Exception as e:
        root.destroy()
        print("导入错误")

def save_model(table):
    print("-------------------------------------------")
    print("以下为表格的预览:")
    print(table.head())
    flag = input("是否需要保存该词频表？(输入Y/N):")
    try:
        if flag == "Y":
            save_file_name = input("请输入该词频表格的名称(不用加.xlsx):")
            save_file_name_xlsx = save_file_name + ".xlsx"
            table.to_excel(save_file_name_xlsx)
            print("存储完毕")
        else:
            print("未能存储")
            pass
    except Exception as e:
        pass

In [10]:
class openlaw_filter:
    def __init__(self,corpus):
        """
        Param corpus:预处理的文本
        """
        self.pattern_list = [] # 初始化规则
        self.corpus = corpus # 初始化预处理文本

    def generate_patterns(self,*args):
        """
        Param *args:需要添加的规则
        return pattern_list:规则表
        """
        self.pattern_list = []
        for item in args[0]:
            pattern_ = re.compile(item)
            self.pattern_list.append(pattern_)
        return self.pattern_list
    
    def basic_mapping(self,single_document):
        """
        Param single_document:文件的单个映射
        """
        try:
            for pattern in self.pattern_list:
                if re.search(pattern,single_document):
                    return True
                else:
                    return False
        except Exception as e:
            return False
        
    def regex_filter(self,*args):
        """
        Param *args:需要添加的规则
        """
        list_args = []
        for item in args:
            list_args.append(item)
        regex_list = self.generate_patterns(list_args)
        result = list(map(self.basic_mapping,self.corpus))
        return result
        
    def find_difference_set(self,single_para,stopwords_clean):
        """
        将stopwords和每一段文本取差集，清洗数据
        param single_para:没清洗文本集中的每一段数据
        return list_clean:每一段清洗好的文本
        """
        list_clean = set(single_para).difference(set(stopwords_clean))
        return list_clean
        
    def data_prepare(self,corpus_clean):
        """
        把数据清理干净
        param text_list:没清洗过的文本列表
        return list_clean:清洗过的文本列表
        """
        table_segments_list = list(map(lambda x:jieba.lcut(x),corpus_clean))
        stopwords = list(map(lambda x:x.strip("\n"),codecs.open('stopwords.txt', 'r', 'utf-8').readlines()))
        stopwords_clean = list(map(lambda x:x.strip("\r"),stopwords))
        list_clean = list(map(self.find_difference_set,table_segments_list,stopwords_clean))
        return list_clean
        
    def tfidf(self,list_clean):
        """ 
        产生tf-idf的模型
        param self.clean_text:清洗过的文本列表
        return corpus_tfidf:tf-idf模型
        """
        texts = list_clean
        # 建立词典
        dictionary = corpora.Dictionary(list_clean)
        
        # 存档词典
        dictionary.save("dict_v1.dict")
        
        # 建立词袋模型
        corpus = [dictionary.doc2bow(text) for text in texts]
        
        # 建立tf-idf模型
        tfidf = models.TfidfModel(corpus)
        
        # 将词袋模型，转换为tf-idf模型
        corpus_tfidf = tfidf[corpus]
        return corpus_tfidf
    
    def find_similar_result(self,corpus_tfidf,article_arg,results):
        """
        使用tf-idf模型，计算文本间的余弦相似，得到结果
        param corpus_tfidf:tf-idf模型
        param target_article_arg:目标文本的下标号
        return list_final_similarities:最终的相似结果
        """
        # 创建索引
        index = similarities.MatrixSimilarity(corpus_tfidf)
        list_final_similarities = []

        # 查找最相似的十个判决结果,以第一个文本为例
        sims = index[corpus_tfidf[article_arg]]
        args = np.argsort(sims)[::-1]
        sims.sort()
        true_sims = sims[::-1]

        final_args = []
        final_args.extend(args)
        for item in list(args):
            list_final_similarities.append(results[item])
        return list_final_similarities,true_sims,args
    
    def replace_none(self,elem):
        if elem == None:
            elem = "None"
        return elem
    
    def cosine_similarity_filter(self,corpus_seq):
        results = list(map(self.replace_none,self.corpus.dropna()))
        list_clean = self.data_prepare(results)
        corpus_tfidf = self.tfidf(list_clean)
        result = self.find_similar_result(corpus_tfidf,corpus_seq,results)
        return result
        
def run_regex_filter(table_final,*args):
    filter_ = openlaw_filter(table_final["法院意见"])
    results = filter_.regex_filter(*args)
    return results
    
def run_cosine_filter(table_filter,corpus_seq):
    filter_ = openlaw_filter(table_filter["判决结果"])
    results,sims,args = filter_.cosine_similarity_filter(corpus_seq)
    results = pd.Series(results)
    sims = pd.Series(sims)
    args = pd.Series(args)
    table_result = pd.DataFrame([sims,args]).T
    table_result["content"] = results
    table_result.columns = ["相似度","文本序号","判决结果内容"]
    return table_result

def regex_filter_of_table(*args):
    root = Tk() # 实例化TKinter窗口
    root.withdraw() # 隐藏TKinter窗口
    table_final = import_table(root)

    results = run_regex_filter(table_final,*args)
    table_final["符合要求的案件"] = results
    table_regex_filter = table_final[table_final["符合要求的案件"] == True]
    return table_regex_filter

In [13]:
## 如果要调用正则表达式筛选器，请在参数部分输入需要查找的参数
import warnings
warnings.filterwarnings('ignore')

if __name__ == "__main__":
    flag_TF = True
    list_regex = []
    while flag_TF:
        regex = input("请输入需要匹配的正则表达式:")
        list_regex.append(regex)
        flag = input("请问还要输入另一个正则表达式吗?(输入Y/N):")
        if flag == "Y":
            flag_TF = True
        else:
            flag_TF = False
    table_regex_result = regex_filter_of_table(*list_regex) # 在这里修改！可以在这里添加结果
    save_model(table_regex_result)

请输入需要匹配的正则表达式:夫妻债务
请问还要输入另一个正则表达式吗?(输入Y/N):Y
请输入需要匹配的正则表达式:共同债务
请问还要输入另一个正则表达式吗?(输入Y/N):N
-------------------------------------------
以下为表格的预览:
                                  标题               案号 案件类型 庭审程序      案由 文书类型  \
0  上诉人官建坤与被上诉人蔡秀英及张兰英民间借贷纠纷一案再审民事判决书    （2018）川10民再3号   民事   再审  民间借贷纠纷  判决书   
1               胡祝荣、刘汉雄民间借贷纠纷再审民事判决书  （2018）鄂1222民再1号   民事   再审  民间借贷纠纷  判决书   
2               胡宇新、魏文祥民间借贷纠纷再审民事判决书   （2018）津01民再15号   民事   再审  民间借贷纠纷  判决书   
4                张薇、何红军民间借贷纠纷再审民事判决书   （2018）川13民再31号   民事   再审  民间借贷纠纷  判决书   
7               陈海宏、於艳娜民间借贷纠纷再审民事判决书    （2018）浙09民再7号   民事   再审  民间借贷纠纷  判决书   

             法院         判决日期   原告       被告   ...   法院意见2  \
0  四川省内江市中级人民法院  2018年05月03日  官建坤  蔡秀英、张兰英   ...     NaN   
1       通城县人民法院  2018年03月21日  胡祝荣  刘汉雄、陈仲华   ...     NaN   
2   天津市第一中级人民法院  2018年03月22日  胡宇新  魏文祥、雷若寒   ...     NaN   
4  四川省南充市中级人民法院  2018年08月03日   张薇  何红军、余宗军   ...     NaN   
7  浙江省舟山市中级人民法院  2018年06月25日  陈海宏   於艳娜、李艳   ...     NaN   

              

In [ ]:
### 请选择使用哪个表？（通过正则处理过的表一般叫做table_filter_regex）
import warnings
warnings.filterwarnings('ignore')

excel_name = input("请选择使用表的表名:(不需要加入.xlsx):")
corpus_seq_num = int(input("请选择使用表中的哪个文本作为相似初始对象？（请输入文本序号）:"))
name_bash = excel_name + ".xlsx"
table_filter_regex = pd.read_excel(name_bash)
table = run_cosine_filter(table_filter_regex,corpus_seq_num)
seq = table["文本序号"]
table_cosine_filter = table_filter_regex.loc[seq]
print("----------------------------------------------------------")
save_to_excel_flag = input("是否要将其存成excel表格:(输入大写的Y或N):")
excel_name = input("请输入该excel表格的名字:")
if save_to_excel_flag == "Y":
    name_bash = excel_name + ".xlsx"
    table_filter_regex.to_excel(name_bash)
elif save_to_excel_flag == "N":
    print("未有存储")
else:
    print("输入错误")
print("--------------------------------------------------------")
print("下列是与第" + str(corpus_seq_num) + "份文书有着相似判决结果的" + str(len(table_cosine_filter)) + "份文书预览")
table_cosine_filter.head()